# 0️⃣ Instruções

*   Importar as bibliotecas, conforme versões informadas no arquivo requirements.txt
*   Para o correto carregamento das 04 bases fornecidas no formato .csv, é necessário que esses arquivos estejam **dentro da mesma pasta deste notebook**.

In [ ]:
# Importação das bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 1️⃣ Análise Exploratória e Limpeza das Bases

In [ ]:
# Carregamento das bases (Windows)
df_cadastro = pd.read_csv("base_cadastral.csv", delimiter=';')
df_info = pd.read_csv("base_info.csv", delimiter=';')
df_pgto = pd.read_csv("base_pagamentos_desenvolvimento.csv", delimiter=';')
df_pgto_teste = pd.read_csv("base_pagamentos_teste.csv", delimiter=';')

## base_cadastral

In [ ]:
# Verificação estrutural
df_cadastro.info()
df_cadastro.head()

In [ ]:
# Verificação da distribuição das categorias
colunas = ['DDD','FLAG_PF','SEGMENTO_INDUSTRIAL','DOMINIO_EMAIL','PORTE','CEP_2_DIG']
for column in colunas:
    print(f"Contagem na coluna '{column}':")
    print(df_cadastro[column].value_counts(dropna=False))
    print("-" * 30)

In [ ]:
# Variável DDD:
# Filtragem de linhas onde o valor da coluna 'DDD' não tem exatamente 2 dígitos
filtro = ~df_cadastro['DDD'].astype(str).str.fullmatch(r'\d{2}')
df_filtrado = df_cadastro.loc[filtro]

# Contagem de ocorrências da coluna 'DDD'
print("Contagem na coluna 'DDD' (valores inválidos):")
print(df_filtrado['DDD'].value_counts(dropna=False))

In [ ]:
# Lista com DDDs fora do padrão, encontrados anteriormente
ddd_inconsistentes = df_cadastro[df_cadastro['DDD'].astype(str).str.startswith('(')]
ddd_nulos = df_cadastro[df_cadastro['DDD'].isnull()]

In [ ]:
# Listas para o preenchimento dos nulos
lista_ceps_ddd_inconsistentes = ddd_inconsistentes['CEP_2_DIG'].values
lista_ceps_ddd_nulo = ddd_nulos['CEP_2_DIG'].values
lista_ceps_final = list(set(ddd_inconsistentes['CEP_2_DIG'].tolist() + ddd_nulos['CEP_2_DIG'].tolist()))

# Dicionário: {'2 primeiros dígitos do CEP': 'DDD correspondente estimado'}
cep_para_ddd = {'66': '66', '96': '96', '77': '77', '55': '55', '21': '21', '36': '35', '28': '27', '32': '32', '26': '24', '98': '98', '48': '48', '78': '65', '63': '63', '23': '24', '95': '95', '16': '16', '62': '62', '89': '89', '40': '85', '91': '91', '83': '83', '81': '81', '50': '67', '15': '15', '87': '87', '88': '48', '90': '11', '29': '71', '12': '12', '43': '43', '59': '84', '14': '16', '61': '61', '35': '35', '57': '82', '84': '84', '56': '56', '97': '97', '54': '54', '20': '21', '22': '22', '13': '19', 'na': 'DDD desconhecido', '65': '65', '17': '17', '99': '98', '85': '85', '49': '49', '37': '37', '75': '75', '46': '46', '68': '68', '93': '93', '33': '22', '70': '61', '27': '27', '58': '83', '71': '71', '72': '62', '34': '34', '76': '77', '53': '53', '18': '18', '42': '42', '64': '64', '79': '79', '73': '73', '60': '61', '45': '45', '19': '19', '30': '31', '11': '11', '69': '69', '80': '11', '47': '47', '38': '38', '25': '24', '41': '41', '86': '86'}

# Preenchimento dos DDDs ausentes com base no CEP
df_cadastro.loc[df_cadastro['DDD'].isnull() | df_cadastro['DDD'].str.startswith('('), 'DDD'] = df_cadastro['CEP_2_DIG'].map(cep_para_ddd)

# Exclusão do registro cujo CEP é 'na', uma vez que não foram encontradas formas de preenchê-lo
df_cadastro = df_cadastro[df_cadastro['CEP_2_DIG'] != 'na']

In [ ]:
# Análise de outliers da variável DDD
sns.histplot(df_cadastro['DDD'])
plt.show()

sns.boxplot(x=df_cadastro['DDD'])
plt.show()

In [ ]:
# Variável DATA_CADASTRO:
# Conversão para data
df_cadastro['DATA_CADASTRO'] = pd.to_datetime(df_cadastro['DATA_CADASTRO'], errors='coerce')

In [ ]:
# Análise de outliers da variável DATA_CADASTRO
sns.histplot(df_cadastro['DATA_CADASTRO'])
plt.show()

sns.boxplot(x=df_cadastro['DATA_CADASTRO'])
plt.show()

In [ ]:
# Análise de outliers da variável CEP_2_DIG
sns.histplot(df_cadastro['CEP_2_DIG'])
plt.show()

sns.boxplot(x=df_cadastro['CEP_2_DIG'])
plt.show()

In [ ]:
# Conversões de CEP_2_DIG e DDD para inteiro, uma vez que estão como números flutuantes
df_cadastro['CEP_2_DIG'] = df_cadastro['CEP_2_DIG'].astype('int64')
df_cadastro['DDD'] = df_cadastro['DDD'].astype('int64')

In [ ]:
# Preenchimento de nulos com "NÃO INFORMADO"
colunas_com_nulos = ['FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE']
df_cadastro[colunas_com_nulos] = df_cadastro[colunas_com_nulos].fillna('NÃO INFORMADO')

In [ ]:
# Cálculo da "idade" do cliente, com base na data de referência de Novembro/2021, que é a data mais recente da base_pagamentos_teste
data_referencia = pd.to_datetime('2021-11-01')
df_cadastro['TEMPO_CADASTRO_ANOS'] = (data_referencia - df_cadastro['DATA_CADASTRO']).dt.days / 365

## base_info

In [ ]:
# Verificação estrutural
df_info.info()
df_info.head()

In [ ]:
# Verificação da distribuição das categorias
for column in df_info:
    print(f"Contagem na coluna '{column}':")
    print(df_info[column].value_counts(dropna=False))
    print("-" * 30)

In [ ]:
# Conversão para data
df_info['SAFRA_REF'] = pd.to_datetime(df_info['SAFRA_REF'].str.strip() + '-01', format='%Y-%m-%d', errors='coerce')

In [ ]:
# Análise da variável SAFRA_REF
sns.histplot(df_info['SAFRA_REF'])

In [ ]:
# Análise da variável RENDA_MES_ANTERIOR
sns.histplot(df_info['RENDA_MES_ANTERIOR'])
plt.show()

sns.boxplot(x=df_info['RENDA_MES_ANTERIOR'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_info['RENDA_MES_ANTERIOR'].quantile(0.25)
Q3 = df_info['RENDA_MES_ANTERIOR'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_info_sem_outliers = df_info.loc[df_info['RENDA_MES_ANTERIOR'].isna() | (df_info['RENDA_MES_ANTERIOR'] <= limite_superior)].copy()

# Verificação
num_removidos = len(df_info) - len(df_info_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_info_sem_outliers.shape}")

# Remoção no dataframe original
df_info = df_info.loc[df_info['RENDA_MES_ANTERIOR'].isna() | (df_info['RENDA_MES_ANTERIOR'] <= limite_superior)]

In [ ]:
# Preenchimento dos nulos
mediana = df_info['RENDA_MES_ANTERIOR'].median()
df_info['RENDA_MES_ANTERIOR'] = df_info['RENDA_MES_ANTERIOR'].fillna(mediana)

In [ ]:
# Análise da variável NO_FUNCIONARIOS
sns.histplot(df_info['NO_FUNCIONARIOS'])
plt.show()

sns.boxplot(x=df_info['NO_FUNCIONARIOS'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_info['NO_FUNCIONARIOS'].quantile(0.25)
Q3 = df_info['NO_FUNCIONARIOS'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_info_sem_outliers = df_info.loc[df_info['NO_FUNCIONARIOS'].isna() | (df_info['NO_FUNCIONARIOS'] >= limite_inferior)].copy()

# Verificação
num_removidos = len(df_info) - len(df_info_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_info_sem_outliers.shape}")

# Remoção no dataframe original
df_info = df_info.loc[df_info['NO_FUNCIONARIOS'].isna() | (df_info['NO_FUNCIONARIOS'] >= limite_inferior)]

In [ ]:
# Preenchimento dos nulos
media = df_info['NO_FUNCIONARIOS'].mean()
df_info['NO_FUNCIONARIOS'] = df_info['NO_FUNCIONARIOS'].fillna(media)

In [ ]:
# Remoção de registros cujo valor de funcionários é zero
df_info = df_info[df_info['NO_FUNCIONARIOS'] != 0]

In [ ]:
# Conversão
df_info['NO_FUNCIONARIOS'] = np.ceil(df_info['NO_FUNCIONARIOS'].astype(float)).astype(int)

## base_pagamentos_desenvolvimento

In [ ]:
# Verificação estrutural
df_pgto.info()
df_pgto.head()

In [ ]:
# Verificação da distribuição das categorias
for column in df_pgto:
    print(f"Contagem na coluna '{column}':")
    print(df_pgto[column].value_counts(dropna=False))
    print("-" * 30)

In [ ]:
# Conversões para data
df_pgto['SAFRA_REF'] = pd.to_datetime(df_pgto['SAFRA_REF'].str.strip() + '-01', format='%Y-%m-%d', errors='coerce')
df_pgto['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_pgto['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto['DATA_PAGAMENTO'] = pd.to_datetime(df_pgto['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto['DATA_VENCIMENTO'] = pd.to_datetime(df_pgto['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Análise das variáveis DATA_PAGAMENTO, DATA_EMISSAO_DOCUMENTO e SAFRA_REF
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sns.boxplot(x=df_pgto['DATA_PAGAMENTO'], ax=axes[0])
sns.boxplot(x=df_pgto['DATA_EMISSAO_DOCUMENTO'], ax=axes[1])
sns.boxplot(x=df_pgto['SAFRA_REF'], ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Análise da variável DATA_VENCIMENTO
sns.histplot(df_pgto['DATA_VENCIMENTO'])
plt.show()

sns.boxplot(x=df_pgto['DATA_VENCIMENTO'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto['DATA_VENCIMENTO'].quantile(0.25)
Q3 = df_pgto['DATA_VENCIMENTO'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_sem_outliers = df_pgto.loc[df_pgto['DATA_VENCIMENTO'].isna() | (df_pgto['DATA_VENCIMENTO'] <= limite_superior)].copy()

# Verificação
num_removidos = len(df_pgto) - len(df_pgto_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto = df_pgto.loc[df_pgto['DATA_VENCIMENTO'].isna() | (df_pgto['DATA_VENCIMENTO'] <= limite_superior)]

In [ ]:
# Análise da variável VALOR_A_PAGAR
sns.histplot(df_pgto['VALOR_A_PAGAR'])
plt.show()

sns.boxplot(x=df_pgto['VALOR_A_PAGAR'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto['VALOR_A_PAGAR'].quantile(0.25)
Q3 = df_pgto['VALOR_A_PAGAR'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_sem_outliers = df_pgto.loc[
    df_pgto['VALOR_A_PAGAR'].isna() | (df_pgto['VALOR_A_PAGAR'] <= limite_superior)].copy()

# Verificação
num_removidos = len(df_pgto) - len(df_pgto_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto = df_pgto.loc[df_pgto['VALOR_A_PAGAR'].isna() | (df_pgto['VALOR_A_PAGAR'] <= limite_superior)]

In [ ]:
# Preenchimento de nulos
mediana = df_pgto['VALOR_A_PAGAR'].median()
df_pgto['VALOR_A_PAGAR'] = df_pgto['VALOR_A_PAGAR'].fillna(mediana)

## base_pagamentos_teste

In [ ]:
# Verificação estrutural
df_pgto_teste.info()
df_pgto_teste.head()

In [ ]:
# Verificação da distribuição das categorias
for column in df_pgto_teste:
    print(f"Contagem na coluna '{column}':")
    print(df_pgto_teste[column].value_counts(dropna=False))
    print("-" * 30)

In [ ]:
# Conversões
df_pgto_teste['SAFRA_REF'] = pd.to_datetime(df_pgto_teste['SAFRA_REF'].astype(str).str.strip() + '-01', errors='coerce')
df_pgto_teste['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_pgto_teste['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto_teste['DATA_VENCIMENTO'] = pd.to_datetime(df_pgto_teste['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Análise da variável SAFRA_REF
sns.histplot(df_pgto_teste['SAFRA_REF'])

In [ ]:
# Análise da variável DATA_EMISSAO_DOCUMENTO
sns.histplot(df_pgto_teste['DATA_EMISSAO_DOCUMENTO'])
plt.show()

sns.boxplot(x=df_pgto_teste['DATA_EMISSAO_DOCUMENTO'])
plt.show()

In [ ]:
# Análise da variável DATA_VENCIMENTO
sns.histplot(df_pgto_teste['DATA_VENCIMENTO'])
plt.show()

sns.boxplot(x=df_pgto_teste['DATA_VENCIMENTO'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto_teste['DATA_VENCIMENTO'].quantile(0.25)
Q3 = df_pgto_teste['DATA_VENCIMENTO'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_teste_sem_outliers = df_pgto_teste.loc[
    df_pgto_teste['DATA_VENCIMENTO'].isna() |
    ((df_pgto_teste['DATA_VENCIMENTO'] >= limite_inferior) &
     (df_pgto_teste['DATA_VENCIMENTO'] <= limite_superior))].copy()

# Verificação
num_removidos = len(df_pgto_teste) - len(df_pgto_teste_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_teste_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto_teste = df_pgto_teste.loc[
    df_pgto_teste['DATA_VENCIMENTO'].isna() |
    ((df_pgto_teste['DATA_VENCIMENTO'] >= limite_inferior) &
     (df_pgto_teste['DATA_VENCIMENTO'] <= limite_superior))]

In [ ]:
# Análise da variável VALOR_A_PAGAR
plt.show()

sns.histplot(x=df_pgto_teste['VALOR_A_PAGAR'])
plt.show()

sns.boxplot(x=df_pgto_teste['VALOR_A_PAGAR'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto_teste['VALOR_A_PAGAR'].quantile(0.25)
Q3 = df_pgto_teste['VALOR_A_PAGAR'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_sem_outliers = df_pgto_teste.loc[df_pgto_teste['VALOR_A_PAGAR'].isna() | (df_pgto_teste['VALOR_A_PAGAR'] <= limite_superior)].copy()

# Verificação
num_removidos = len(df_pgto_teste) - len(df_pgto_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto_teste = df_pgto_teste.loc[df_pgto_teste['VALOR_A_PAGAR'].isna() | (df_pgto_teste['VALOR_A_PAGAR'] <= limite_superior)]

In [ ]:
# Preenchimento de nulos
mediana = df_pgto_teste['VALOR_A_PAGAR'].median()
df_pgto_teste['VALOR_A_PAGAR'] = df_pgto_teste['VALOR_A_PAGAR'].fillna(mediana)

# 2️⃣ Análise, Modelagem e Preparação Final

## Enriquecimento da base_pagamentos_desenvolvimento limpa, com as features das outras bases também limpas

In [ ]:
# Conversões para data
# df_info
df_info['SAFRA_REF'] = pd.to_datetime(df_info['SAFRA_REF'], format='%Y-%m-%d', errors='coerce')

# df_pgto
df_pgto['SAFRA_REF'] = pd.to_datetime(df_pgto['SAFRA_REF'], format='%Y-%d-%m', errors='coerce')
df_pgto['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_pgto['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto['DATA_PAGAMENTO'] = pd.to_datetime(df_pgto['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto['DATA_VENCIMENTO'] = pd.to_datetime(df_pgto['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto['PK_CLIENTE_SAFRA'] = df_pgto['ID_CLIENTE'].astype(str) + '_' + df_pgto['SAFRA_REF'].astype(str).astype(str).str[:7]

In [ ]:
# Criação de features

# Dias de atraso no pagamento
df_pgto['ATRASO'] = (df_pgto['DATA_PAGAMENTO'] - df_pgto['DATA_VENCIMENTO']).dt.days

# Tempo entre emissão da nota e vencimento
df_pgto['PRAZO_CREDITO_DIAS'] = (df_pgto['DATA_VENCIMENTO'] - df_pgto['DATA_EMISSAO_DOCUMENTO']).dt.days

In [ ]:
# Agrupamento - adequação da granularidade de datas
df_pgto_agrupado = df_pgto.groupby('PK_CLIENTE_SAFRA', as_index=False)[['VALOR_A_PAGAR', 'ATRASO', 'TAXA','PRAZO_CREDITO_DIAS']].mean()
df_pgto_agrupado['ID_CLIENTE'] = df_pgto_agrupado['PK_CLIENTE_SAFRA'].str.split('_').str[0]
df_pgto_agrupado['SAFRA_REF'] = df_pgto_agrupado['PK_CLIENTE_SAFRA'].str.split('_').str[1]
df_pgto_agrupado['ID_CLIENTE'] = df_pgto_agrupado['ID_CLIENTE'].astype(int)

# Enriquecimento
# LEFT JOIN com df_info
df_info['PK_CLIENTE_SAFRA'] = df_info['ID_CLIENTE'].astype(str) + '_' + df_info['SAFRA_REF'].dt.strftime('%Y-%m')
df_pgto_agrupado = df_pgto_agrupado.merge(df_info[['PK_CLIENTE_SAFRA', 'RENDA_MES_ANTERIOR','NO_FUNCIONARIOS']], on='PK_CLIENTE_SAFRA', how='left')

# LEFT JOIN com df_cadastro
df_pgto_agrupado = df_pgto_agrupado.merge(df_cadastro, on='ID_CLIENTE', how='left')

# Criação do target
df_pgto_agrupado['INADIMPLENTE'] = (df_pgto_agrupado['ATRASO'] >= 5).astype(int)

## Tratamento da base de modelagem agrupada e enriquecida

In [ ]:
# Remoção de colunas
df_pgto_agrupado.drop(columns=['DATA_CADASTRO'], inplace=True)

# Preenchimento de nulos
df_pgto_agrupado['DDD'] = df_pgto_agrupado['DDD'].fillna('11')
df_pgto_agrupado['FLAG_PF'] = df_pgto_agrupado['FLAG_PF'].fillna('NÃO INFORMADO')
df_pgto_agrupado['SEGMENTO_INDUSTRIAL'] = df_pgto_agrupado['SEGMENTO_INDUSTRIAL'].fillna('NÃO INFORMADO')
df_pgto_agrupado['DOMINIO_EMAIL'] = df_pgto_agrupado['DOMINIO_EMAIL'].fillna('NÃO INFORMADO')
df_pgto_agrupado['PORTE'] = df_pgto_agrupado['PORTE'].fillna('NÃO INFORMADO')
df_pgto_agrupado['CEP_2_DIG'] = df_pgto_agrupado['CEP_2_DIG'].fillna('NÃO INFORMADO')
df_pgto_agrupado['TEMPO_CADASTRO_ANOS'] = df_pgto_agrupado['TEMPO_CADASTRO_ANOS'].fillna(df_pgto_agrupado['TEMPO_CADASTRO_ANOS'].median())

In [ ]:
# Análise da variável NO_FUNCIONARIOS
sns.histplot(df_pgto_agrupado['NO_FUNCIONARIOS'])
plt.show()

sns.boxplot(x=df_pgto_agrupado['NO_FUNCIONARIOS'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto_agrupado['NO_FUNCIONARIOS'].quantile(0.25)
Q3 = df_pgto_agrupado['NO_FUNCIONARIOS'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_teste_sem_outliers = df_pgto_agrupado.loc[
    df_pgto_agrupado['NO_FUNCIONARIOS'].isna() |
    ((df_pgto_agrupado['NO_FUNCIONARIOS'] >= limite_inferior) &
     (df_pgto_agrupado['NO_FUNCIONARIOS'] <= limite_superior))].copy()

# Verificação
num_removidos = len(df_pgto_agrupado) - len(df_pgto_teste_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_teste_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto_agrupado = df_pgto_agrupado.loc[
    df_pgto_agrupado['NO_FUNCIONARIOS'].isna() |
    ((df_pgto_agrupado['NO_FUNCIONARIOS'] >= limite_inferior) &
     (df_pgto_agrupado['NO_FUNCIONARIOS'] <= limite_superior))]

In [ ]:
# Análise da variável RENDA_MES_ANTERIOR
sns.histplot(df_pgto_agrupado['RENDA_MES_ANTERIOR'])
plt.show()

sns.boxplot(x=df_pgto_agrupado['RENDA_MES_ANTERIOR'])
plt.show()

In [ ]:
# Cálculo dos quartis e criação dos limites
Q1 = df_pgto_agrupado['RENDA_MES_ANTERIOR'].quantile(0.25)
Q3 = df_pgto_agrupado['RENDA_MES_ANTERIOR'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Filtragem do dataframe para cálculo da redução de registros
df_pgto_sem_outliers = df_pgto_agrupado.loc[df_pgto_agrupado['RENDA_MES_ANTERIOR'].isna() | (df_pgto_agrupado['RENDA_MES_ANTERIOR'] <= limite_superior)].copy()

# Verificação
num_removidos = len(df_pgto_agrupado) - len(df_pgto_sem_outliers)
print(f"Registros removidos: {num_removidos}")
print(f"Novo tamanho: {df_pgto_sem_outliers.shape}")

# Remoção no dataframe original
df_pgto_agrupado = df_pgto_agrupado.loc[df_pgto_agrupado['RENDA_MES_ANTERIOR'].isna() | (df_pgto_agrupado['RENDA_MES_ANTERIOR'] <= limite_superior)]

In [ ]:
# Preenchimendo dos valores ausentes de renda e funcionários com a mediana
df_pgto_agrupado['RENDA_MES_ANTERIOR'] = df_pgto_agrupado['RENDA_MES_ANTERIOR'].fillna(df_pgto_agrupado['RENDA_MES_ANTERIOR'].median())
df_pgto_agrupado['NO_FUNCIONARIOS'] = df_pgto_agrupado['NO_FUNCIONARIOS'].fillna(df_pgto_agrupado['NO_FUNCIONARIOS'].median())

# Conversão do tipo de dado do target, para adequação ao modelo
df_pgto_agrupado['INADIMPLENTE'] = df_pgto_agrupado['INADIMPLENTE'].astype(bool)

# 3️⃣ Treinamento, Aplicação e Avaliação do Modelo

## Ajustes finais da base e treinamento do modelo

In [ ]:
# Cópia do dataset, para maior controle e praticidade
df = df_pgto_agrupado.copy()

# Extração do ano e mês como inteiros
df['SAFRA_ANO'] = df['SAFRA_REF'].str.slice(0, 4).astype(int)
df['SAFRA_MES'] = df['SAFRA_REF'].str.slice(5, 7).astype(int)

# Remoção de colunas que não contribuirão para o treinamento do modelo
df.drop(columns=['PK_CLIENTE_SAFRA', 'ID_CLIENTE', 'DOMINIO_EMAIL', 'DDD', 'SAFRA_REF', 'ATRASO'], inplace=True)

# CEP_2_DIG: redução da quantidade de categorias, para manter apenas as mais frequentes
top_10_cep = df['CEP_2_DIG'].value_counts().nlargest(10).index
df['CEP_2_DIG'] = df['CEP_2_DIG'].where(df['CEP_2_DIG'].isin(top_10_cep), 'OUTROS')
df['CEP_2_DIG'] = df['CEP_2_DIG'].astype(str).str.replace(r'\.0$', '', regex=True)

# Ajuste de valores de FLAG_PF
df['FLAG_PF'] = df['FLAG_PF'].map({'X': 1, 'NÃO INFORMADO': 0}).astype(bool)

# Encoding manual de SEGMENTO_INDUSTRIAL
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL'], drop_first=True)
df.drop(columns=['SEGMENTO_INDUSTRIAL_NÃO INFORMADO'], inplace=True)

# Encoding manual de PORTE
porte_map = {'PEQUENO': 1, 'MEDIO': 2, 'GRANDE': 3, 'NÃO INFORMADO': 0}
df['PORTE'] = df['PORTE'].map(porte_map)

# Separação do target
X = df.drop('INADIMPLENTE', axis=1)
y = df['INADIMPLENTE']

# Definição das listas de colunas
colunas_onehot = ['CEP_2_DIG']
colunas_sacler = ['VALOR_A_PAGAR', 'PRAZO_CREDITO_DIAS', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS', 'TEMPO_CADASTRO_ANOS']
colunas_ignorar = ['TAXA', 'FLAG_PF', 'PORTE','SAFRA_MES','SAFRA_ANO','SEGMENTO_INDUSTRIAL_Indústria', 'SEGMENTO_INDUSTRIAL_Serviços']

# Montagem do ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # colunas que sofrerão One‑Hot
        ('ohe_seg',   OneHotEncoder(sparse_output=False, handle_unknown='ignore'), colunas_onehot),
        # colunas numéricas que sofrerão Scaler
        ('scale_num', StandardScaler(), colunas_sacler),
        # Pass‑through das colunas que não mudam
        ('pass',      'passthrough', colunas_ignorar),
],
    remainder='drop'
)

# Instanciamento do Pipeline
pipeline = ImbPipeline([
    ('preprocessador', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('modelo', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# Separação e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

## Avaliação de performance do modelo

In [ ]:
# Comparação da acurácia entre dataset de treino e de teste
acuracia_treino = pipeline.score(X_train, y_train)
acuracia_teste  = pipeline.score(X_test,  y_test)
print(f"Acurácia base de treino: {acuracia_treino:.4f}")
print(f"Acurácia base de teste: {acuracia_teste:.4f}\n")

In [ ]:
# Teste de validação cruzada
cv_scores = cross_val_score(
    pipeline, X, y, cv=5, scoring='accuracy', n_jobs=-1
    )

print('CV scores:      ', cv_scores)
print(f"Média CV:       {cv_scores.mean():.4f}")
print(f"Desvio‑padrão:  {cv_scores.std():.4f}\n")

In [ ]:
# Curva de aprendizado: evolução do desempenho do modelo em função do volume de dados de treino
train_sizes, train_scores, val_scores = learning_curve(
    pipeline, X, y, train_sizes=np.linspace(0.1, 1.0, 5), cv=5, scoring='accuracy', n_jobs=-1
    )
train_mean = train_scores.mean(axis=1)
val_mean   = val_scores.mean(axis=1)

print('Resultados curva de aprendizado:')
for n, tr, va in zip(train_sizes, train_mean, val_mean):
    pct = n / X.shape[0]
    print(f"  Treino {n} amostras ({pct:.0%}): treino={tr:.4f}, validação={va:.4f}")
print()

In [ ]:
# Matriz de confusão e Relatório de classificação
y_pred = pipeline.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Matriz de confusão:')
print(cm)
print('\nRelatório de classificação:')
print(classification_report(y_test, y_pred))

In [ ]:
# Curva ROC e cálculo da área sob a curva (AUC)
probs = pipeline.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
auc = roc_auc_score(y_test, probs)

# Plotagem
plt.plot(fpr, tpr)
plt.plot([0,1], [0,1], '--')
plt.xlabel('Taxa de falsos positivos')
plt.ylabel('Taxa de verdadeiros positivos')
plt.title(f'Curva ROC: área sob a curva (AUC) = {auc:.4f}')
plt.show()

In [ ]:
# Estatística de Kolmogorov-Smirnov (KS)
y_proba = pipeline.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
ks_statistic = max(tpr - fpr)
ks_threshold = thresholds[np.argmax(tpr - fpr)]

print(f'Estatística KS: {ks_statistic:.4f}')
print(f'Threshold correspondente: {ks_threshold:.4f}')

# 4️⃣ Geração de previsões na base base_pagamentos_teste

## Adequação da base de teste

In [ ]:
# Conversões para data
# df_pgto_teste
df_pgto_teste['SAFRA_REF'] = pd.to_datetime(df_pgto_teste['SAFRA_REF'], format='%Y-%d-%m', errors='coerce')
df_pgto_teste['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_pgto_teste['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce')
df_pgto_teste['DATA_VENCIMENTO'] = pd.to_datetime(df_pgto_teste['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce')

# df_info
df_info['SAFRA_REF'] = pd.to_datetime(df_info['SAFRA_REF'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Criação de features e agrupamentos
df_pgto_teste['PK_CLIENTE_SAFRA'] = df_pgto_teste['ID_CLIENTE'].astype(str) + '_' + df_pgto_teste['SAFRA_REF'].astype(str).astype(str).str[:7]

# Tempo entre emissão da nota e vencimento
df_pgto_teste['PRAZO_CREDITO_DIAS'] = (df_pgto_teste['DATA_VENCIMENTO'] - df_pgto_teste['DATA_EMISSAO_DOCUMENTO']).dt.days

# Agrupamento - adequação da granularidade de datas
df_pgto_teste_agrupado = df_pgto_teste.groupby('PK_CLIENTE_SAFRA', as_index=False)[['VALOR_A_PAGAR', 'TAXA','PRAZO_CREDITO_DIAS']].mean()
df_pgto_teste_agrupado['ID_CLIENTE'] = df_pgto_teste_agrupado['PK_CLIENTE_SAFRA'].str.split('_').str[0]
df_pgto_teste_agrupado['SAFRA_REF'] = df_pgto_teste_agrupado['PK_CLIENTE_SAFRA'].str.split('_').str[1]
df_pgto_teste_agrupado['ID_CLIENTE'] = df_pgto_teste_agrupado['ID_CLIENTE'].astype(int)

# Enriquecimento
# LEFT JOIN com df_info e df_cadastro
df_info['PK_CLIENTE_SAFRA'] = df_info['ID_CLIENTE'].astype(str) + '_' + df_info['SAFRA_REF'].dt.strftime('%Y-%m')

df_pgto_teste_agrupado = df_pgto_teste_agrupado.merge(df_info[['PK_CLIENTE_SAFRA', 'RENDA_MES_ANTERIOR','NO_FUNCIONARIOS']], on='PK_CLIENTE_SAFRA', how='left')
df_pgto_teste_agrupado = df_pgto_teste_agrupado.merge(df_cadastro, on='ID_CLIENTE', how='left')

In [ ]:
# Remoção de colunas
df_pgto_teste_agrupado.drop(columns=['DATA_CADASTRO'], inplace=True)

# Preenchimento de nulos
df_pgto_teste_agrupado['DDD'] = df_pgto_teste_agrupado['DDD'].fillna('11')
df_pgto_teste_agrupado['FLAG_PF'] = df_pgto_teste_agrupado['FLAG_PF'].fillna('NÃO INFORMADO')
df_pgto_teste_agrupado['SEGMENTO_INDUSTRIAL'] = df_pgto_teste_agrupado['SEGMENTO_INDUSTRIAL'].fillna('NÃO INFORMADO')
df_pgto_teste_agrupado['DOMINIO_EMAIL'] = df_pgto_teste_agrupado['DOMINIO_EMAIL'].fillna('NÃO INFORMADO')
df_pgto_teste_agrupado['PORTE'] = df_pgto_teste_agrupado['PORTE'].fillna('NÃO INFORMADO')
df_pgto_teste_agrupado['CEP_2_DIG'] = df_pgto_teste_agrupado['CEP_2_DIG'].fillna('NÃO INFORMADO')
df_pgto_teste_agrupado['TEMPO_CADASTRO_ANOS'] = df_pgto_teste_agrupado['TEMPO_CADASTRO_ANOS'].fillna(df_pgto_teste_agrupado['TEMPO_CADASTRO_ANOS'].median())

# Extração de ano e mês como inteiros
df_pgto_teste_agrupado['SAFRA_ANO'] = df_pgto_teste_agrupado['SAFRA_REF'].str.slice(0, 4).astype(int)
df_pgto_teste_agrupado['SAFRA_MES'] = df_pgto_teste_agrupado['SAFRA_REF'].str.slice(5, 7).astype(int)

In [ ]:
# Cópia do dataframe, para maior controle e praticidade
df = df_pgto_teste_agrupado.copy()

# Extração do ano e mês como inteiros
df['SAFRA_ANO'] = df['SAFRA_REF'].str.slice(0, 4).astype(int)
df['SAFRA_MES'] = df['SAFRA_REF'].str.slice(5, 7).astype(int)

# Remoção de colunas que não contribuirão para o treinamento do modelo
df.drop(columns=['PK_CLIENTE_SAFRA', 'DOMINIO_EMAIL', 'DDD', 'SAFRA_REF'], inplace=True)

# CEP_2_DIG: redução da quantidade de categorias, para manter apenas as mais frequentes
top_10_cep = df['CEP_2_DIG'].value_counts().nlargest(10).index
df['CEP_2_DIG'] = df['CEP_2_DIG'].where(df['CEP_2_DIG'].isin(top_10_cep), 'OUTROS')

# Ajuste do tipo de dado
df['FLAG_PF'] = df['FLAG_PF'].map({'X': 1, 'NÃO INFORMADO': 0}).astype(bool)

# One-Hot Encoding para SEGMENTO_INDSUTRIAL
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL'], drop_first=True)
df.drop(columns=['SEGMENTO_INDUSTRIAL_NÃO INFORMADO'], inplace=True)

# Label Encoding manual para controlar qual valor numérico cada categoria receberá
porte_map = {'PEQUENO': 1, 'MEDIO': 2, 'GRANDE': 3, 'NÃO INFORMADO': 0}
df['PORTE'] = df['PORTE'].map(porte_map)

## Geração de previsões

In [ ]:
df_pgto_teste_randomforest = df.copy()

In [ ]:
# Previsão das classes (0 ou 1)
y_pred = pipeline.predict(df_pgto_teste_randomforest)

# Previsão das probabilidades das classes
y_proba = pipeline.predict_proba(df_pgto_teste_randomforest)[:, 1]

# Adição ao dataframe
df_pgto_teste_randomforest['PREDITO_INADIMPLENTE'] = y_pred
df_pgto_teste_randomforest['PROBABILIDADE_INADIMPLENCIA'] = y_proba
df_pgto_teste_randomforest['SAFRA_REF'] = df_pgto_teste_randomforest['SAFRA_ANO'].astype(str) + '-' + \
    np.where(df_pgto_teste_randomforest['SAFRA_MES'].isin([10, 11, 12]),
             df_pgto_teste_randomforest['SAFRA_MES'].astype(str),
             '0' + df_pgto_teste_randomforest['SAFRA_MES'].astype(str))

# Visualizar os primeiros resultados
df_pgto_teste_randomforest['PROBABILIDADE_INADIMPLENCIA'].head()

In [ ]:
# Exportação do arquivo final
df_pgto_teste_randomforest[['ID_CLIENTE','SAFRA_REF','PROBABILIDADE_INADIMPLENCIA']].to_csv('submissao_case.csv', index=False)